In [1]:
import pandas as pd
import re

# прочитаем все файлы
df1 = pd.read_csv('1.tsv', sep='\t')  # 1) Советский период
df2 = pd.read_csv('2.tsv', sep='\t')  # 2) 1990-е годы
df3 = pd.read_csv('3.tab', sep='\t')  # 3) Постсоветский период 

# включим уникальный номер для каждого датасета для облегчения иденттификации
df1['uid']='1_'+df1.index.astype(str)
df2['uid']='2_'+df2.index.astype(str)
df3['uid']='3_'+df3.index.astype(str)

# ручные правки
df2.loc[df2['uid']==2_10615,'author (автор)']=''
df2.loc[df2['uid']==2_10615,'summary (примечание)']='пер. А.С. Пушкина'
df2.loc[df2['uid']==2_12698,'author (автор)']=''
df2.loc[df2['uid']==2_12698,'summary (примечание)']='Сост. В. Аникин; Б. Шергин'
df2.loc[df2['uid']==2_12881,'author (автор)']=''
df2.loc[df2['uid']==2_12881,'summary (примечание)']='Сост. В.П. Аникин'
df2.loc[df2['uid']==2_12883,'author (автор)']=''
df2.loc[df2['uid']==2_12883,'summary (примечание)']='Сост. Ю.П. Круглов'

# сделаем четкий и универсальный признак обязательности 0 и 1
df1['oblig'] = df1.apply(lambda row: 0 if row['priority'] == '*' else 1, axis=1)   # устанавливаем 0 для *... астериском помечены необязательные
df2['oblig'] = df2.apply(lambda row: 1 if row['priority (обязательное или нет)'] == '' else 0, axis=1)   # если ['priority (обязательное или нет)'] == '' то 1  
df3['oblig'] = df3.apply(lambda row: 1 if (row['oblig_title'] == 'да') else 0, axis=1) # если ['oblig_title'] == 'да' то 1  

# переименуем под единые названия author, title, grade, year, в первом датасете уже все хорошо
df2.columns = ['author', 'title', 'comment (раздел)', 'curriculum','id', 'year', 'grade','priority (обязательное или нет)', 'first publication', 'quantity','summary (примечание)', 'uid', 'oblig']
df3.columns = ['source_id', 'year', 'group', 'century', 'author', 'title',
       'genre', 'type', 'oblig_author', 'oblig_title', 'grade', 'section',
       'number_authors', 'number_titles', 'profound', 'source_issue',
       'title_cycle', 'notes', 'uid', 'oblig']

# выбросим то, что не читается по годам (1937-38) и оставим только свой период для каждого датасета
df1 = df1[pd.to_numeric(df1['year'], errors='coerce').notnull()]
df1 = df1[(df1['year'].astype(int) >= 1974) & (df1['year'].astype(int) <= 1991)]
df2 = df2[pd.to_numeric(df2['year'], errors='coerce').notnull()]
df2 = df2[(df2['year'].astype(int) >= 1992) & (df2['year'].astype(int) <= 1997)]
df3 = df3[pd.to_numeric(df3['year'], errors='coerce').notnull()]
df3 = df3[(df3['year'].astype(int) >= 1998) & (df3['year'].astype(int) <= 2022)]

# пока оставим только 6 полей для обработкти  uid, author, title, grade, year, oblig - остальные поля подключим после объедиения через uid
columns_req = ['uid','author', 'title', 'grade', 'year', 'oblig']
concatenated_df = pd.concat([df1[columns_req], df2[columns_req], df3[columns_req]], ignore_index=True)

# экспериментальная часть с инициалами - в конечном итоге выбрали ручную чистку авторов
def replace_capital_letter_with_space(s):
    out = re.sub(r'([А-Я])\.', r' ', s)
    out = re.sub(r'\s', '', out)
    out = out.lower()
    return out
#concatenated_df['author'] = concatenated_df['author'].astype(str).apply(lambda x: replace_capital_letter_with_space(x))
concatenated_df.to_csv('concatenated.csv', index = False)

#сохраним в grade_mistakes некорректные значения
concatenated_df[pd.to_numeric(concatenated_df['grade'], errors='coerce').isna()].to_csv('grade_mistakes.csv', index = False)

# выведем по комбинации 'author' +  'title'  все вхождения uid
result_df = concatenated_df.groupby(['author', 'title'])['uid'].agg(lambda x: ', '.join(map(str, x))).reset_index()
result_df.rename(columns={'group': 'result'}, inplace=True)
result_df.to_csv('grouped.csv', index = False)

len(result_df)  # 4241


4241

In [43]:
import re
import pandas as pd

# перечитаем concatenated_5.csv с ручными правками 
concatenated_df5 =  pd.read_csv('concatenated_5.csv', sep=';')

# заменим Фамилия И.О. на Фамилия И. О. — с пробелом между инициалами
def add_space_after_capital_and_dot(s):
    # Use a regular expression to add a space after capital letter and dot (if not last symbol)
    out = re.sub(r'([А-Я])\.(?=[^. ])', r'\1. ', s)
    return out.rstrip()

# отбросим финальные пробелы
def remove_space(s):
    return s.rstrip()
    
concatenated_df5['author'] = concatenated_df5['author'].astype(str).apply(lambda x: add_space_after_capital_and_dot(x))
concatenated_df5['title'] = concatenated_df5['title'].astype(str).apply(lambda x: remove_space(x))
    
# выведем по комбинации 'author' +  'title'  все вхождения uid
result_df5 = concatenated_df5.groupby(['author', 'title'])['uid'].agg(lambda x: ', '.join(map(str, x))).reset_index()
result_df5.rename(columns={'group': 'result'}, inplace=True)
result_df5.to_csv('grouped5.csv', index = False)

len(result_df5) # 3536

3536

In [5]:
df2.loc[df2['uid']=='2_11142']

,author,title,comment (раздел),curriculum,id,year,grade,priority (обязательное или нет),first publication,quantity,summary (примечание),uid,oblig
11142,Андреев Л.Н.,Иуда Искариот,русская,NaN,NaN,1992,11,самостоятельно,NaN,не найден,NaN,2_11142,0
